## **Translation**

### **Sequence-to-Sequence Task**

- Dịch máy là bài toán chuyển đổi từ chuỗi này sang chuỗi khác → **giống tóm tắt văn bản**.
- Các bài toán tương tự khác:
  - **Style transfer**: chuyển đổi phong cách viết.
  - **Generative QA**: sinh câu trả lời từ ngữ cảnh.

---

### **Hai hướng tiếp cận chính**

| Cách làm | Mô tả |
|---------|-------|
| **Huấn luyện từ đầu** | Nếu có **corpus lớn song ngữ**, bạn có thể huấn luyện một mô hình dịch mới hoàn toàn. |
| **Fine-tune mô hình có sẵn** | Tiết kiệm thời gian, hiệu quả hơn. Có thể fine-tune: <br> - Mô hình đa ngôn ngữ như **mT5**, **mBART** <br> - Mô hình chuyên biệt cho cặp ngôn ngữ như **MarianMT** |

---

### **Thí nghiệm: Fine-tune MarianMT trên bộ dữ liệu KDE4**

- **MarianMT**: Mô hình chuyên dịch, pretrained với dữ liệu từ bộ **Opus**.
- **KDE4**: Bộ dữ liệu gồm các tập tin đã được dịch cho ứng dụng KDE (giàu nội dung kỹ thuật).
- Dù MarianMT đã thấy dữ liệu này trong pretraining, **fine-tune vẫn giúp tăng hiệu năng** cho domain cụ thể.

---

### **Tại sao nên fine-tune?**

- Mô hình gốc học tổng quát → chưa tối ưu cho ngữ cảnh hẹp (domain-specific).
- Fine-tuning giúp mô hình **tập trung và chuẩn hóa hơn với cách dùng từ, ngữ pháp đặc trưng** trong lĩnh vực cụ thể (vd. giao diện phần mềm KDE).

---

### **Preparing the Data**
Để fine-tune hoặc huấn luyện một mô hình dịch thuật từ đầu, ta cần một tập dữ liệu song ngữ — nghĩa là mỗi câu trong một ngôn ngữ phải đi kèm với bản dịch của nó trong ngôn ngữ kia. Dataset **KDE4** là một ví dụ điển hình, thường được dùng cho các cặp ngôn ngữ như tiếng Anh và tiếng Đức.

In [1]:
from datasets import load_dataset

raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")

raw_datasets

README.md:   0%|          | 0.00/5.10k [00:00<?, ?B/s]

kde4.py:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

In [3]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [4]:
split_datasets["train"][1]["translation"]

{'en': 'Default to expanded threads',
 'fr': 'Par défaut, développer les fils de discussion'}

Dataset này cung cấp các cặp câu song ngữ theo ngôn ngữ yêu cầu. Tuy nhiên, có một điểm đặc biệt: mặc dù đây là dữ liệu chuyên ngành kỹ thuật máy tính và được dịch sang tiếng Pháp đầy đủ, nhưng nhiều thuật ngữ kỹ thuật như “threads” thường được giữ nguyên tiếng Anh trong thực tế. Trong khi đó, dataset này lại dịch thành “fils de discussion”. Mô hình pretrained mà ta sử dụng — vốn đã học trên tập dữ liệu lớn chứa cả tiếng Pháp và tiếng Anh — thường chọn cách đơn giản hơn: giữ nguyên từ gốc tiếng Anh.

### **Processing the data**
1. **Tạo tokenizer**  
   Sử dụng `AutoTokenizer` từ Hugging Face để tải tokenizer tương ứng với mô hình pretrained dịch Anh → Pháp
   
   Nếu bạn dùng tokenizer đa ngôn ngữ (như mBART, M2M100), cần thiết lập `tokenizer.src_lang` và `tokenizer.tgt_lang`.

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


2. **Token hóa input và target**  
   Phải **token hóa cả câu nguồn và câu đích**. Dữ liệu đầu vào (tiếng Anh) và đích (tiếng Pháp) được đưa qua `tokenizer` với `text_target` để xử lý đúng:
   
   ⚠️ Nếu bạn quên `text_target`, tokenizer sẽ xử lý target như câu tiếng Anh, gây ra lỗi tokenization do không nhận diện từ vựng tiếng Pháp.

In [6]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [47591, 12, 9842, 19634, 9, 0], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [577, 5891, 2, 3184, 16, 2542, 5, 1710, 0]}

In [7]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Par', '▁dé', 'f', 'aut', ',', '▁dé', 've', 'lop', 'per', '▁les', '▁fil', 's', '▁de', '▁discussion', '</s>']
['▁Par', '▁défaut', ',', '▁développer', '▁les', '▁fils', '▁de', '▁discussion', '</s>']


3. **Hàm tiền xử lý dữ liệu**  

In [8]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

4. **Áp dụng tiền xử lý lên dataset**  

5. **Lưu ý thêm:**  
   - Với mô hình **T5**, cần thêm prefix vào input: `"translate English to French: <text>"`
   - Không cần attention mask cho target.
   - Padding token trong labels phải chuyển thành `-100` để không ảnh hưởng khi tính loss (được xử lý tự động nếu bạn dùng data collator với dynamic padding).

In [9]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/189155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21018 [00:00<?, ? examples/s]

#### **Data collation**

Để huấn luyện mô hình dịch máy, ta cần xử lý padding một cách phù hợp cho cả **inputs** và **labels**. Đây là lý do ta sử dụng `DataCollatorForSeq2Seq` thay vì `DataCollatorWithPadding`.

---

1. Vì sao không dùng `DataCollatorWithPadding`?

- `DataCollatorWithPadding` chỉ pad `input_ids`, `attention_mask` (và `token_type_ids` nếu có).
- Với mô hình Seq2Seq, **labels cũng cần được padding**.
- Giá trị padding của labels phải là `-100`, để tránh ảnh hưởng đến loss khi tính toán.

---

2. Sử dụng `DataCollatorForSeq2Seq`

- `model` được truyền vào vì collator sẽ tạo `decoder_input_ids`, tức là phiên bản dịch được **shift** sang phải với một token đặc biệt ở đầu.
- Cách shift khác nhau tuỳ kiến trúc mô hình nên cần truyền vào `model`.

In [12]:
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

### **Metrics**

#### **Tại sao dùng `Seq2SeqTrainer`?**

Khác với `Trainer` thông thường, `Seq2SeqTrainer` hỗ trợ:
- Tự động gọi `generate()` khi đánh giá (`predict_with_generate=True`), giúp mô phỏng quá trình inference thực tế.
- Tự xử lý các bước đặc biệt như `decoder_input_ids`, `attention mask`,…

---

#### **Dùng BLEU và SacreBLEU**

BLEU là metric phổ biến cho bài toán dịch, nhưng có hạn chế vì cần **token hóa trước**.  
→ Thay vào đó, ta dùng **SacreBLEU**, một phiên bản chuẩn hóa, dễ so sánh hơn.

---

In [15]:
%%capture
!pip install sacrebleu evaluate

In [16]:
import evaluate

metric = evaluate.load("sacrebleu")

predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [17]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

### **Fine-tuning the model**

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"marian-finetuned-kde4-en-to-fr",
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False, # change to True if you want push to hub
)

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_31/543751272.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.evaluate(max_length=max_length) # check score before train

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(max_length=max_length)

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")